In [1]:
import pandas as pd
path_future = "C:\\PICKLE\\BX Future"
path_options = "C:\\PICKLE\\BX Options"
fut_path = "2024-06-05_bankex_future.pkl"
opt_path = "2024-06-05_bankex.pkl"
opt = pd.read_pickle(path_options + "\\" + opt_path).set_index("date_time")
fut = pd.read_pickle(path_future + "\\" + fut_path).set_index("date_time")
opt.index = pd.to_datetime(opt.index)
fut.index = pd.to_datetime(fut.index)
opt["date_time"] = opt.index
fut["date_time"] = fut.index

In [2]:
entry_time = pd.to_datetime("2024-06-05 09:16:00")
exit_time = pd.to_datetime("2024-06-05 13:00:00")

In [14]:
from BacktestOptions import IntradayBacktest
bt = IntradayBacktest("C:\\PICKLE\\", "NIFTY", entry_time,3,entry_time.time(),exit_time.time())
bt.get_strangle_strike(entry_time, exit_time, 0.5)
bt = IntradayBacktest(
    "C:\\PICKLE\\", "bankex", entry_time, 3, entry_time.time(), exit_time.time()
)
# print(bt.get_strangle_strike(entry_time, exit_time, 1))
bt.get_straddle_strike(entry_time, exit_time, 0.3)

2024-06-05 09:16:00
future price 53748.1
53700CE 53700PE
2024-06-05 09:17:00
future price 53712.2
53700CE 53700PE
2024-06-05 09:18:00
future price 53616.89
53600CE 53600PE
2024-06-05 09:19:00
future price 53610.17
53600CE 53600PE
2024-06-05 09:20:00
future price 53470.31
53500CE 53500PE
2024-06-05 09:21:00
future price 53642.12
53600CE 53600PE
2024-06-05 09:22:00
future price 53570.72
53600CE 53600PE
2024-06-05 09:23:00
future price 53405.86
53400CE 53400PE
2024-06-05 09:24:00
future price 53324.62
53300CE 53300PE
2024-06-05 09:25:00
future price 53345.92
53300CE 53300PE
2024-06-05 09:26:00
future price 53458.81
53500CE 53500PE
2024-06-05 09:27:00
future price 53566.74
53600CE 53600PE
2024-06-05 09:28:00
future price 53634.53
53600CE 53600PE
2024-06-05 09:29:00
future price 53567.53
53600CE 53600PE
2024-06-05 09:30:00
future price 53737.97
53700CE 53700PE
2024-06-05 09:31:00
future price 53878.7
53900CE 53900PE
2024-06-05 09:32:00
future price 53786.7
53800CE 53800PE
2024-06-05 09:33:0

('54800CE',
 '54000PE',
 np.float64(456.45),
 np.float64(445.9),
 np.float64(54417.64),
 Timestamp('2024-06-05 11:16:00'))

In [13]:
def get_straddle_strike(opt:pd.DataFrame,fut:pd.DataFrame,start_dt:pd.Timedelta, end_dt:pd.Timedelta,gap:int=100, sd:int=0, SDroundoff:bool=False):

    valid_times=fut.loc[start_dt:end_dt].index
    for current_dt in valid_times:
        # sd_range = 0
        try:
            print(current_dt)
            future_price=fut.loc[current_dt,'close']
            print("future price",future_price)
            round_future_price = int(round(future_price/gap)*gap)
            ce_scrip, pe_scrip = f"{round_future_price}CE", f"{round_future_price}PE"
            print("ce scrip, pe scrip",ce_scrip,pe_scrip)
            ce_price,pe_price = opt.loc[(opt["date_time"] == current_dt)&(opt['scrip'] == ce_scrip),'close'].iloc[0],opt.loc[(opt["date_time"] == current_dt)&(opt['scrip'] == pe_scrip),'close'].iloc[0]
            print(ce_price,pe_price)
            syn_future = ce_price-pe_price+round_future_price
            round_syn_future = round(syn_future/gap)*gap
            ce_scrip_list=[f"{round_syn_future}CE",f"{round_syn_future+gap}CE",f"{round_syn_future-gap}CE"]
            pe_scrip_list=[f"{round_syn_future}PE",f"{round_syn_future+gap}PE",f"{round_syn_future-gap}PE"]

            scrip_index, min_value = None, float("inf")

            for i in range(3):
                try:
                    ce_price = opt[
                        (opt.index == current_dt) & (opt["scrip"] == ce_scrip_list[i])
                    ].close.iloc[0]
                    pe_price = opt[
                        (opt.index == current_dt) & (opt["scrip"] == pe_scrip_list[i])
                    ].close.iloc[0]

                    diff = abs(ce_price-pe_price)
                    if min_value>diff:
                        min_value=diff
                        scrip_index=i

                except :
                    pass
            # print(pe_scrip_list)
            ce_scrip,pe_scrip=ce_scrip_list[scrip_index], pe_scrip_list[scrip_index]
            ce_price,pe_price = opt.loc[(opt["date_time"] == current_dt)&(opt['scrip'] == ce_scrip),'close'].iloc[0],opt.loc[(opt["date_time"] == current_dt)&(opt['scrip'] == pe_scrip),'close'].iloc[0]

            if sd:
                sd_range = (ce_price+pe_price)*sd

                if SDroundoff:
                    sd_range = round(sd_range/gap)*gap
                else:
                    sd_range = max(gap,round(sd_range/gap)*gap)

                ce_scrip, pe_scrip = (
                    f"{int(ce_scrip[:-2]) + sd_range}CE",
                    f"{int(pe_scrip[:-2]) - sd_range}PE",
                )
            ce_price, pe_price=opt.loc[(opt["date_time"] == current_dt)&(opt['scrip'] == ce_scrip),'close'].iloc[0],opt.loc[(opt["date_time"] == current_dt)&(opt['scrip'] == pe_scrip),'close'].iloc[0]

            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt

        except Exception as e :
            print(e)
            print("error occured in get straddle strike")
            continue

    return None,None,None,None,None,None
get_straddle_strike(opt,fut,entry_time,exit_time,sd = 0.3)

2024-06-05 09:16:00
future price 53748.1
ce scrip, pe scrip 53700CE 53700PE
single positional indexer is out-of-bounds
error occured in get straddle strike
2024-06-05 09:17:00
future price 53712.2
ce scrip, pe scrip 53700CE 53700PE
single positional indexer is out-of-bounds
error occured in get straddle strike
2024-06-05 09:18:00
future price 53616.89
ce scrip, pe scrip 53600CE 53600PE
single positional indexer is out-of-bounds
error occured in get straddle strike
2024-06-05 09:19:00
future price 53610.17
ce scrip, pe scrip 53600CE 53600PE
single positional indexer is out-of-bounds
error occured in get straddle strike
2024-06-05 09:20:00
future price 53470.31
ce scrip, pe scrip 53500CE 53500PE
single positional indexer is out-of-bounds
error occured in get straddle strike
2024-06-05 09:21:00
future price 53642.12
ce scrip, pe scrip 53600CE 53600PE
single positional indexer is out-of-bounds
error occured in get straddle strike
2024-06-05 09:22:00
future price 53570.72
ce scrip, pe scrip

('54800CE',
 '54000PE',
 np.float64(456.45),
 np.float64(445.9),
 np.float64(54417.64),
 Timestamp('2024-06-05 11:16:00'))

In [23]:
def get_one_om(fut,future_price=None,STEP=5000):
    future_price = fut['close'].iloc[0] if future_price is None else future_price
    return (int(future_price/STEP)*STEP)/100

In [25]:
def get_strangle_strike(fut,start_time,end_time,opt,gap=100,om=None,target=None,check_inverted=False, tf=1):
    valid_times = fut.loc[start_time:end_time].index
    for current_dt in valid_times:
        try:
            future_price = fut.loc[current_dt,'close']
            # future price = 17510.4
            one_om = get_one_om(fut,future_price)
            # one om = 170.0
            target = one_om*om if target is None else target
            # target = 68.0
            target_od = opt[(opt.index==current_dt) & (opt['close'] >= target*tf)].sort_values(by=['close']).copy()

            ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
            pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'), 'scrip'].iloc[0]
            # ce_scrip = 17550CE 
            # pe_scrip = 17400PE
            
            ce_scrip_list = [ce_scrip, f"{int(ce_scrip[:-2])-gap}CE", f"{int(ce_scrip[:-2])+gap}CE"]
            pe_scrip_list = [pe_scrip, f"{int(pe_scrip[:-2])-gap}PE", f"{int(pe_scrip[:-2])+gap}PE"]
            # ce_scrip_list = ['17550CE', '17500CE', '17600CE']
            # pe_scrip_list = ['17400PE', '17350PE', '17450PE']
            
            call_list_prices, put_list_prices = [], []
            for z in range(3):
                try:
                    call_list_prices.append(opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip_list[z])]['close'].iloc[0])
                except:
                    call_list_prices.append(0)
                    print("call list price is empty and my date time column is not present in your data")
                try:
                    put_list_prices.append(opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip_list[z])]['close'].iloc[0])
                except:
                    put_list_prices.append(0)
                    print("put list price is empty and my date time column is not present in your data")
            
            call, put, min_diff = call_list_prices[0], put_list_prices[0], float('inf')
            target_2, target_3 = target*2*tf, target*3
            diff = abs(put-call)
            
            required_call, required_put = None, None
            if (put+call >= target_2) & (min_diff > diff) & (put+call <= target_3):
                min_diff = diff
                required_call, required_put = call, put  
                
            for i in range(1,3):
                if (min_diff > abs(put_list_prices[i] - call)) & (put_list_prices[i]+call >= target_2) & (put_list_prices[i]+call <= target_3):
                    min_diff = abs(put_list_prices[i] - call)
                    required_call, required_put = call, put_list_prices[i]

                if (min_diff > abs(call_list_prices[i] - put)) & (call_list_prices[i]+put >= target_2) & (call_list_prices[i]+put <= target_3):
                    min_diff = abs(call_list_prices[i] - put)
                    required_call, required_put = call_list_prices[i], put


            ce_scrip, pe_scrip = ce_scrip_list[call_list_prices.index(required_call)], pe_scrip_list[put_list_prices.index(required_put)]
            ce_price, pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip)]['close'].iloc[0], opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip)]['close'].iloc[0]
            if int(ce_scrip[:-2]) < int(pe_scrip[:-2]) and check_inverted:
                get_straddle_strike(fut,current_dt,end_time,opt)
            else:
                return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt

        except Exception as e  :
            print("error occured in get strangle strike",e)
            continue
    
    return None, None, None, None, None, None
get_strangle_strike(fut,entry_time,exit_time,opt,om=1)


error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike single positional indexer is out-of-bounds
error occured in get strangle strike sin

('81900CE',
 '83500PE',
 np.float64(950.0),
 np.float64(1000.0),
 np.float64(82585.53),
 Timestamp('2024-09-02 09:29:00'))

In [8]:
def get_strangle_strike(opt:pd.DataFrame,fut:pd.DataFrame,start_dt:pd.Timestamp, end_dt:pd.Timestamp,gap:int=100, om:float=None):
    """
    Args:
        opt (pd.DataFrame): [description]
        fut (pd.DataFrame): [description]
        start_dt (pd.Timedelta): [description]
        end_dt (pd.Timedelta): [description]
        gap (int, optional): [description]. Defaults to 50.
        om (float, optional): [description]. Defaults to None.
    """
    valid_times=fut.loc[start_dt:end_dt].index
    # lat's take example
    for current_dt in valid_times:
        try:
            future_price=fut.loc[current_dt,'close']
            # future price = 17510.4 at current date and close
            one_om = get_one_om(fut, future_price)
            # one om = 170.0
            target = one_om*om
            # target = 170.0 * 0.4  lats take om=0.4
            # target = 68.0
            
            # keep data only with index current dt and ehose close is greater target
            target_od = opt[(opt.index==current_dt) & (opt['close'] >= target)].sort_values(by=['close']).copy()
            
            ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
            pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'), 'scrip'].iloc[0]
            # ce_scrip = 17550CE 
            # pe_scrip = 17400PE
            
            ce_scrip_list = [ce_scrip, ce_scrip, ce_scrip, f"{int(ce_scrip[:-2])-gap}CE", f"{int(ce_scrip[:-2])+gap}CE"]
            pe_scrip_list = [pe_scrip, f"{int(pe_scrip[:-2])-gap}PE", f"{int(pe_scrip[:-2])+gap}PE", pe_scrip, pe_scrip]
            
            # ce_scrip_list ['17550CE', '17550CE', '17550CE', '17500CE', '17600CE']
            # pe_scrip_list ['17400PE', '17350PE', '17450PE', '17400PE', '17400PE']
            # take each pair and compair min difference of price of scrip
            print(ce_scrip_list, pe_scrip_list)
            
            ce_price_list,pe_price_list= [],[]
            
            for i in range(5):
                try:
                    ce_price_list.append(opt[(opt.index == current_dt) & (opt['scrip']==ce_scrip_list[i])]['close'].iloc[0])
                except :
                    ce_price_list.append(0)
                    print(f"price not found at {current_dt} in ce_scrip {ce_scrip_list[i]}")
                try:
                    pe_price_list.append(opt[(opt.index == current_dt) & (opt['scrip']==pe_scrip_list[i])]['close'].iloc[0])
                except:
                    pe_price_list.append(0)
                    print(f"price not found at {current_dt} in pe_scrip {pe_scrip_list[i]}  ")
            # ce price list contain prices at scrip 
            # ce_price_list [np.float64(68.3), np.float64(68.3), np.float64(68.3), np.float64(92.25), np.float64(49.4)]
            # pe_price_list [np.float64(79.65), np.float64(64.2), np.float64(99.1), np.float64(79.65), np.float64(79.65)]
            target_2, target_3 = target*2, target*3
            # target_2 = 136.0 =  68 * 2
            # target_3 =204.0 =  68 * 3
            
            min_diff = float('inf')
            scrip_index = None

            for i in range(5):
                if (min_diff > abs(ce_price_list[i]-pe_price_list[i])) and (ce_price_list[i] + pe_price_list[i] >= target_2) and (ce_price_list[i] + pe_price_list[i] <= target_3) :
                    min_diff = abs(ce_price_list[i]-pe_price_list[i])
                    scrip_index = i
            
            # min_diff = 11.35
            
            ce_scrip,pe_scrip = ce_scrip_list[scrip_index] , pe_scrip_list[scrip_index]
            # ce_scrip 17550CE 
            # pe_scrip 17400PE
            ce_price,pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip)]['close'].iloc[0], opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip)]['close'].iloc[0]
            # ce_price 68.3 
            # pe_price 79.65
            
            # return this  ('17550CE', '17400PE', 68.3, 79.65, 17510.4, '2022-01-03 09:20:00')
            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt
        
        except Exception as e:
            print(f"error occured in get strangle {e}")
            continue
    return None,None,None, None,None,None   #else return None
get_strangle_strike(opt,fut,entry_time,exit_time,om=0.1)

['73600CE', '73600CE', '73600CE', '73500CE', '73700CE'] ['70400PE', '70300PE', '70500PE', '70400PE', '70400PE']
price not found at 2024-01-01 09:16:00 in pe_scrip 70300PE  


('73600CE',
 '70500PE',
 np.float64(74.65),
 np.float64(73.6),
 np.float64(72087.38),
 Timestamp('2024-01-01 09:16:00'))

In [ ]:
def get_strangle_strike(opt:pd.DataFrame,fut:pd.DataFrame,start_dt:pd.Timestamp, end_dt:pd.Timestamp,gap:int=50, om:float=None):
    """
    Args:
        opt (pd.DataFrame): [description]
        fut (pd.DataFrame): [description]
        start_dt (pd.Timedelta): [description]
        end_dt (pd.Timedelta): [description]
        gap (int, optional): [description]. Defaults to 50.
        om (float, optional): [description]. Defaults to None.
    """
    valid_times=fut.loc[start_dt:end_dt].index
    # lates take example
    for current_dt in valid_times:
        try:
            future_price=fut.loc[current_dt,'close']
            # future price = 17510.4
            one_om = get_one_om(fut, future_price)
            # one om = 170.0
            target = one_om*om
            target_od = opt[(opt.index==current_dt) & (opt['close'] >= target)].sort_values(by=['close']).copy()
            
            ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
            pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'), 'scrip'].iloc[0]
            ce_scrip_list = [ce_scrip, ce_scrip, ce_scrip, f"{int(ce_scrip[:-2])-gap}CE", f"{int(ce_scrip[:-2])+gap}CE"]
            pe_scrip_list = [pe_scrip, f"{int(pe_scrip[:-2])-gap}PE", f"{int(pe_scrip[:-2])+gap}PE", pe_scrip, pe_scrip]
            
            ce_price_list,pe_price_list= [],[]
            for i in range(5):
                try:
                    ce_price_list.append(opt[(opt['date_time'] == current_dt) & (opt['scrip']==ce_scrip_list[i])]['close'].iloc[0])
                except :
                    ce_scrip_list.append(0)
                    print("price not found")
                try:
                    pe_price_list.append(opt[(opt['date_time'] == current_dt) & (opt['scrip']==pe_scrip_list[i])]['close'].iloc[0])
                except:
                    pe_price_list.append(0)
                    print("price not found")
            target_2, target_3 = target*2, target*3
            min_diff = float('inf')
            scrip_index = None

            for i in range(5):
                if (min_diff > abs(ce_price_list[i]-pe_price_list[i])) and (ce_price_list[i] + pe_price_list[i] >= target_2) and (ce_price_list[i] + pe_price_list[i] <= target_3) :
                    min_diff = abs(ce_price_list[i]-pe_price_list[i])
                    scrip_index = i
            ce_scrip,pe_scrip = ce_scrip_list[scrip_index] , pe_scrip_list[scrip_index]
            ce_price,pe_price = opt[(opt['date_time'] == current_dt) & (opt['scrip'] == ce_scrip)]['close'].iloc[0], opt[(opt['date_time'] == current_dt) & (opt['scrip'] == pe_scrip)]['close'].iloc[0]
            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt
        except Exception as e:
            print(f"error occured in get strangle {e}")
            continue
    return None,None,None, None,None,None
get_strangle_strike(opt,fut,entry_time,exit_time,om=0.5)

In [ ]:
def get_strangle_strike(
    opt: pd.DataFrame,
    fut: pd.DataFrame,
    start_dt: pd.Timestamp,
    end_dt: pd.Timestamp,
    gap: int = 50,
    om: float = None,
):
    """
    Args:
        opt (pd.DataFrame): [description]
        fut (pd.DataFrame): [description]
        start_dt (pd.Timedelta): [description]
        end_dt (pd.Timedelta): [description]
        gap (int, optional): [description]. Defaults to 50.
        om (float, optional): [description]. Defaults to None.
    """
    valid_times = fut.loc[start_dt:end_dt].index
    # lat's take example
    for current_dt in valid_times:
        try:
            future_price = fut.loc[current_dt, "close"]
            one_om = get_one_om(fut, future_price)
            target = one_om * om
            
            target_od = (
                opt[(opt.index == current_dt) & (opt["close"] >= target)]
                .sort_values(by=["close"])
                .copy()
            )

            ce_scrip = target_od.loc[
                target_od["scrip"].str.endswith("CE"), "scrip"
            ].iloc[0]
            pe_scrip = target_od.loc[
                target_od["scrip"].str.endswith("PE"), "scrip"
            ].iloc[0]
            ce_scrip_list = [
                ce_scrip,
                ce_scrip,
                ce_scrip,
                f"{int(ce_scrip[:-2])-gap}CE",
                f"{int(ce_scrip[:-2])+gap}CE",
            ]
            pe_scrip_list = [
                pe_scrip,
                f"{int(pe_scrip[:-2])-gap}PE",
                f"{int(pe_scrip[:-2])+gap}PE",
                pe_scrip,
                pe_scrip,
            ]

            ce_price_list, pe_price_list = [], []
            for i in range(5):
                try:
                    ce_price_list.append(
                        opt[
                            (opt["date_time"] == current_dt)
                            & (opt["scrip"] == ce_scrip_list[i])
                        ]["close"].iloc[0]
                    )
                except:
                    ce_scrip_list.append(0)
                    print(f"price not found for {current_dt} and {ce_scrip_list[i]} in ce scrip")
                try:
                    pe_price_list.append(
                        opt[
                            (opt["date_time"] == current_dt)
                            & (opt["scrip"] == pe_scrip_list[i])
                        ]["close"].iloc[0]
                    )
                except:
                    pe_price_list.append(0)
                    print(f"price not found for {current_dt} and {pe_scrip_list[i]} in pe scrip")

            target_2, target_3 = target * 2, target * 3

            min_diff = float("inf")
            scrip_index = None

            for i in range(5):
                if (
                    (min_diff > abs(ce_price_list[i] - pe_price_list[i]))
                    and (ce_price_list[i] + pe_price_list[i] >= target_2)
                    and (ce_price_list[i] + pe_price_list[i] <= target_3)
                ):
                    min_diff = abs(ce_price_list[i] - pe_price_list[i])
                    scrip_index = i

            ce_scrip, pe_scrip = ce_scrip_list[scrip_index], pe_scrip_list[scrip_index]
            ce_price, pe_price = (
                opt[(opt["date_time"] == current_dt) & (opt["scrip"] == ce_scrip)][
                    "close"
                ].iloc[0],
                opt[(opt["date_time"] == current_dt) & (opt["scrip"] == pe_scrip)][
                    "close"
                ].iloc[0],
            )
            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt

        except Exception as e:
            print(f"error occured in get strangle {e}")
            continue
    return None, None, None, None, None, None  # else return None
get_strangle_strike(opt, fut, entry_time, exit_time, om=0.5)


In [53]:
get_straddle_strike(opt,fut,entry_time,exit_time)

TypeError: unsupported operand type(s) for -: 'Categorical' and 'Categorical'

In [49]:
get_straddle_strike(opt,fut,entry_time,exit_time)

unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorical'
error occured in get straddle strike
unsupported operand type(s) for -: 'Categorical' and 'Categorica

(None, None, None, None, None, None)

In [37]:
om=0.2
valid_times = fut.loc[entry_time:exit_time].index
future_price = fut.loc[valid_times[3],'close']
one_om = get_one_om(fut,future_price)
one_om
target = one_om*om 
target_od = opt[(opt["date_time"]==valid_times[3]) & (opt['close'] >= target)].sort_values(by=['close']).copy()
target_od


,index,scrip,open,high,low,close,volume,openinterest,expiry_date,dte,date_time
date_time,,,,,,,,,,,
